In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('clean_Movie5')

In [3]:
df

,Unnamed: 0,sentiment,Reviews3
0,0,Positive,stuff going moment mj started listening music ...
1,1,Positive,nothe classic war worlds timothy hines enterta...
2,2,Negative,starts manager nicholas bell giving welcome in...
3,3,Negative,must assumed praised nothe greatest filmed ope...
4,4,Positive,superbly trashy wondrously unpretentious explo...
...,...,...,...
24995,24995,Negative,seems like consideration gone imdb reviews wen...
24996,24996,Negative,not believe made completely unnecessary first ...
24997,24997,Negative,guy loser not get girls needs build picked str...
24998,24998,Negative,minute documentary buñuel made early spain poo...


In [4]:
df['Label'] = df['sentiment'].apply(lambda x: '1' if x == 'Positive' else ('0' if x == 'Negative' else 'neutral'))

In [5]:
df.drop(['Unnamed: 0','sentiment'],inplace=True,axis=1)

In [6]:
df

,Reviews3,Label
0,stuff going moment mj started listening music ...,1
1,nothe classic war worlds timothy hines enterta...,1
2,starts manager nicholas bell giving welcome in...,0
3,must assumed praised nothe greatest filmed ope...,0
4,superbly trashy wondrously unpretentious explo...,1
...,...,...
24995,seems like consideration gone imdb reviews wen...,0
24996,not believe made completely unnecessary first ...,0
24997,guy loser not get girls needs build picked str...,0
24998,minute documentary buñuel made early spain poo...,0


In [15]:
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from prettytable import PrettyTable
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score,accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np


In [8]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

In [9]:
tokenizer = LemmaTokenizer()


In [10]:
df['Tokens'] = df['Reviews3'].apply(tokenizer)

In [11]:
df

,Reviews3,Label,Tokens
0,stuff going moment mj started listening music ...,1,"[stuff, going, moment, mj, started, listening,..."
1,nothe classic war worlds timothy hines enterta...,1,"[nothe, classic, war, world, timothy, hines, e..."
2,starts manager nicholas bell giving welcome in...,0,"[start, manager, nicholas, bell, giving, welco..."
3,must assumed praised nothe greatest filmed ope...,0,"[must, assumed, praised, nothe, greatest, film..."
4,superbly trashy wondrously unpretentious explo...,1,"[superbly, trashy, wondrously, unpretentious, ..."
...,...,...,...
24995,seems like consideration gone imdb reviews wen...,0,"[seems, like, consideration, gone, imdb, revie..."
24996,not believe made completely unnecessary first ...,0,"[not, believe, made, completely, unnecessary, ..."
24997,guy loser not get girls needs build picked str...,0,"[guy, loser, not, get, girl, need, build, pick..."
24998,minute documentary buñuel made early spain poo...,0,"[minute, documentary, buñuel, made, early, spa..."


In [12]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [13]:
from gensim.models import Word2Vec

#skip-gram
modelw = Word2Vec(sentences=df['Tokens'], vector_size=100, window=5, min_count=1, workers=4)

df['Review_Embedding'] = df['Tokens'].apply(
    lambda tokens: sum(modelw.wv[token] for token in tokens if token in modelw.wv)
)

df

,Reviews3,Label,Tokens,Review_Embedding
0,stuff going moment mj started listening music ...,1,"[stuff, going, moment, mj, started, listening,...","[-61.522743, 98.165985, 43.87292, 49.077736, -..."
1,nothe classic war worlds timothy hines enterta...,1,"[nothe, classic, war, world, timothy, hines, e...","[-7.450311, 27.769035, 18.905077, 15.507267, 2..."
2,starts manager nicholas bell giving welcome in...,0,"[start, manager, nicholas, bell, giving, welco...","[-68.85421, 52.91655, 25.118242, -19.980772, 5..."
3,must assumed praised nothe greatest filmed ope...,0,"[must, assumed, praised, nothe, greatest, film...","[-77.65243, 46.693996, 37.906998, 37.29187, -1..."
4,superbly trashy wondrously unpretentious explo...,1,"[superbly, trashy, wondrously, unpretentious, ...","[-78.60378, 65.34709, -2.7407186, 20.885601, -..."
...,...,...,...,...
24995,seems like consideration gone imdb reviews wen...,0,"[seems, like, consideration, gone, imdb, revie...","[-18.400814, 17.781273, 4.7585373, 19.116325, ..."
24996,not believe made completely unnecessary first ...,0,"[not, believe, made, completely, unnecessary, ...","[-41.02454, 44.665936, 32.92913, 22.206049, -2..."
24997,guy loser not get girls needs build picked str...,0,"[guy, loser, not, get, girl, need, build, pick...","[-17.119638, 37.17709, -2.2863996, 9.641801, -..."
24998,minute documentary buñuel made early spain poo...,0,"[minute, documentary, buñuel, made, early, spa...","[-30.271288, 34.879288, 19.87128, 15.452813, -..."


In [16]:


# Prepare features (embeddings) and labels
x = np.array(df['Review_Embedding'].tolist())  # Embeddings as feature matrix
y = np.array(df['Label'])  # Sentiment labels

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [17]:

lmodel = LogisticRegression()
lmodel.fit(x_train, y_train)

y_pred = lmodel.predict(x_test)

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
print("Accuracy on test dataset: %s" % accuracy_score(y_test,lmodel.predict(x_test)))
print("Precision Score on test: %s" % precision_score(y_test,lmodel.predict(x_test),average='micro'))
print("AUC Score on test: %s" % roc_auc_score(y_test,lmodel.predict_proba(x_test)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,lmodel.predict(x_test),average="weighted")
print("F1 Score: %s" % f1_score_4)

Accuracy on test dataset: 0.8536
Precision Score on test: 0.8536
AUC Score on test: 0.9215564290993448
F1 Score: 0.8535655534870275


In [21]:

rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)

In [26]:
print("Accuracy on train dataset for Random forest classifier: %s" % accuracy_score(y_train,rf_model.predict(x_train)))
print("Precision Score on training dateset for Random Forest Classifier: %s" % precision_score(y_train,rf_model.predict(x_train),average='micro'))
print("AUC Score on training dateset for Random Forest Classifier: %s" % roc_auc_score(y_train,rf_model.predict_proba(x_train)[:,1],multi_class='ovo',average='macro'))
f1_score_train_4 =f1_score(y_train,rf_model.predict(x_train),average="weighted")
print("F1 Score training dateset for Random Forest Classifier: %s" % f1_score_train_4)
print("Accuracy on test dataset for Random forest classifier: %s" % accuracy_score(y_test,rf_model.predict(x_test)))
print("Precision Score on test for Random Forest Classifier: %s" % precision_score(y_test,rf_model.predict(x_test),average='micro'))
print("AUC Score on test for Random Forest Classifier: %s" % roc_auc_score(y_test,rf_model.predict_proba(x_test)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,rf_model.predict(x_test),average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_4)

Accuracy on train dataset for Random forest classifier: 1.0
Precision Score on training dateset for Random Forest Classifier: 1.0
AUC Score on training dateset for Random Forest Classifier: 1.0
F1 Score training dateset for Random Forest Classifier: 1.0
Accuracy on test dataset for Random forest classifier: 0.8168
Precision Score on test for Random Forest Classifier: 0.8168
AUC Score on test for Random Forest Classifier: 0.8957168566056375
F1 Score for Random Forest Classifier: 0.8167056648527239


In [25]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(x_train, y_train)

# Predict on training data
y_train_pred = dt_model.predict(x_train)


In [29]:
print("Accuracy on train dataset: %s" % accuracy_score(y_train,dt_model.predict(x_train)))
print("Precision Score on training dateset: %s" % precision_score(y_train,dt_model.predict(x_train),average='micro'))
print("AUC Score on training dateset: %s" % roc_auc_score(y_train,dt_model.predict_proba(x_train)[:,1],multi_class='ovo',average='macro'))
f1_score_train_4 =f1_score(y_train,dt_model.predict(x_train),average="weighted")
print("F1 Score training dateset: %s" % f1_score_train_4)
print("Accuracy on test dataset: %s" % accuracy_score(y_test,dt_model.predict(x_test)))
print("Precision Score on test: %s" % precision_score(y_test,dt_model.predict(x_test),average='micro'))
print("AUC Score on test: %s" % roc_auc_score(y_test,dt_model.predict_proba(x_test)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,dt_model.predict(x_test),average="weighted")
print("F1 Score: %s" % f1_score_4)

Accuracy on train dataset: 1.0
Precision Score on training dateset: 1.0
AUC Score on training dateset: 1.0
F1 Score training dateset: 1.0
Accuracy on test dataset: 0.7072
Precision Score on test: 0.7072
AUC Score on test: 0.707143564612292
F1 Score: 0.707180508038437
